In [1]:
#import libraries

import pandas as pd
import numpy as np
import seaborn as sb
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

from sklearn.cross_validation import train_test_split
from pandas import Series, DataFrame
from pylab import rcParams


from sklearn.metrics import classification_report,accuracy_score
sb.set(style="white")
sb.set(style="whitegrid", color_codes=True)
import itertools
from sklearn import svm
import graphviz
from sklearn.model_selection import GridSearchCV,StratifiedKFold, LeaveOneOut
from sklearn.datasets import make_regression

import warnings; warnings.simplefilter('ignore')

/home/sahulphaniraj/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
### reading data and printing the data ###
data = pd.read_csv('/home/sahulphaniraj/csce633-ml/MachineLearningProject/Dataset/classical_feature.csv',names=['ID','TruncatedID','Mean','SD','AUC','Peak','MaxIndex','Halfpk','Skew','HHMean','Fat','Carb','Protein'
])
print(data.shape)
print(list(data.columns))

(171, 13)
['ID', 'TruncatedID', 'Mean', 'SD', 'AUC', 'Peak', 'MaxIndex', 'Halfpk', 'Skew', 'HHMean', 'Fat', 'Carb', 'Protein']


In [3]:
### CHO
CHO_data = data.drop(['ID','Protein','Carb'], axis=1) #taking only the Carb target column
print(CHO_data.Fat.unique())# Finding the unique values in that column
print(CHO_data.TruncatedID.unique()) #finiding unique IDs

[20.  18.   2.5]
['2133-002' '2133-022' '2133-020' '2133-004' '2133-010' '2133-015'
 '2133-040' '2133-013' '2133-008' '2133-019' '2133-001' '2133-024'
 '2133-011' '2133-018' '2133-006' '2133-032' '2133-025' '2133-017'
 '2133-028' '2133-039' '2133-021' '2133-030' '2133-033' '2133-037'
 '2133-026' '2133-012' '2133-036' '2133-035' '2133-041' '2133-009']


In [4]:
###for LOW-HIGH

CHO_LH_data = CHO_data[CHO_data['Fat'].isin([2.5,20.0])] #considering the two classes for binary classification
CHO_LH_data.loc[CHO_LH_data.Fat == 2.5, 'CHO_bin'] = 0 #giving them binary values, as double variables sometimes create error
CHO_LH_data.loc[CHO_LH_data.Fat== 20.0, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Fat'], axis=1)
#the above 4 lines will be different for different classification, the remaining code will be same
X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009'] #needed for manual loss

final_accuracy =[] #will be used for finding mean across nested cross validation

for i in range(0,30): #as we have 30 IDs
    print("For fold :", i+1) #will print the current fold number, this is for initial check, can delete it
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])] #test data    
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]#train data
    X_train = train.loc[:, train.columns != 'CHO_bin'] #test feature
    Y_train = train.loc[:, train.columns == 'CHO_bin'] #test label
    X_test = test.loc[:, test.columns != 'CHO_bin']#train feature    
    Y_test =test.loc[:, test.columns == 'CHO_bin']#train label
    
    train_ids = X_train['TruncatedID']#needed for inner cross-validation
    train_ids = np.array(train_ids)#needed for inner cross-validation
    train_ids = np.unique(train_ids)#needed for inner cross-validation
    X_train = X_train.drop(['TruncatedID'],axis = 1) #dropping this column as it is not a feature for classification
    X_test = X_test.drop(['TruncatedID'],axis = 1)#dropping this column as it is not a feature for classification
    max_acc = 0 #initializing maximum accuracy which will be needed later    
    ypred_all = []
    for C_val in range(1,20): #ranges of decision tree depth
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification        
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            clf = svm.SVC(C=C_val) #SVM classifier
            #convert dataframe to array
            #xtr_arr = xtr.as_matrix(columns=None)
            #ytr_arr = ytr.as_matrix(columns=None)
            #clf = clf.fit(xtr_arr,ytr_arr)
            clf=clf.fit(xtr,ytr)
            ypred = clf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = C_val
    #best SVM classifier
    print(best_param)
    clf = svm.SVC(C=best_param) 
    clf=clf.fit(X_train,Y_train)
    Y_pred = clf.predict(X_test)
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies in outer CV)


For fold : 1
14
For fold : 2
10
For fold : 3
17
For fold : 4
11
For fold : 5
10
For fold : 6
11
For fold : 7
12
For fold : 8
12
For fold : 9
13
For fold : 10
12
For fold : 11
14
For fold : 12
11
For fold : 13
14
For fold : 14
10
For fold : 15
19
For fold : 16
17
For fold : 17
15
For fold : 18
11
For fold : 19
10
For fold : 20
10
For fold : 21
11
For fold : 22
14
For fold : 23
13
For fold : 24
10
For fold : 25
13
For fold : 26
11
For fold : 27
13
For fold : 28
13
For fold : 29
13
For fold : 30
12
The final accuracy is :  0.7833333333333333


In [5]:
#MEDIUM-HIGH

CHO_LH_data = CHO_data[CHO_data['Fat'].isin([18.0,20.0])] #considering the two classes for binary classification
CHO_LH_data.loc[CHO_LH_data.Fat == 18.0, 'CHO_bin'] = 0 #giving them binary values, as double variables sometimes create error
CHO_LH_data.loc[CHO_LH_data.Fat== 20.0, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Fat'], axis=1)
#the above 4 lines will be different for different classification, the remaining code will be same
X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009'] #needed for manual loss

final_accuracy =[] #will be used for finding mean across nested cross validation

for i in range(0,30): #as we have 30 IDs
    print("For fold :", i+1) #will print the current fold number, this is for initial check, can delete it
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])] #test data    
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]#train data
    X_train = train.loc[:, train.columns != 'CHO_bin'] #test feature
    Y_train = train.loc[:, train.columns == 'CHO_bin'] #test label
    X_test = test.loc[:, test.columns != 'CHO_bin']#train feature    
    Y_test =test.loc[:, test.columns == 'CHO_bin']#train label
    
    train_ids = X_train['TruncatedID']#needed for inner cross-validation
    train_ids = np.array(train_ids)#needed for inner cross-validation
    train_ids = np.unique(train_ids)#needed for inner cross-validation
    X_train = X_train.drop(['TruncatedID'],axis = 1) #dropping this column as it is not a feature for classification
    X_test = X_test.drop(['TruncatedID'],axis = 1)#dropping this column as it is not a feature for classification
    max_acc = 0 #initializing maximum accuracy which will be needed later    
    ypred_all = []
    for C_val in range(1,20): #ranges of decision tree depth
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification        
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            clf = svm.SVC(C=C_val) #SVM classifier
            #convert dataframe to array
            #xtr_arr = xtr.as_matrix(columns=None)
            #ytr_arr = ytr.as_matrix(columns=None)
            #clf = clf.fit(xtr_arr,ytr_arr)
            clf=clf.fit(xtr,ytr)
            ypred = clf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = C_val
    #best SVM classifier
    print(best_param)
    clf = svm.SVC(C=best_param) 
    clf=clf.fit(X_train,Y_train)
    Y_pred = clf.predict(X_test)
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies in outer CV)



For fold : 1
5
For fold : 2
6
For fold : 3
4
For fold : 4
8
For fold : 5
5
For fold : 6
4
For fold : 7
4
For fold : 8
4
For fold : 9
4
For fold : 10
4
For fold : 11
4
For fold : 12
10
For fold : 13
4
For fold : 14
4
For fold : 15
5
For fold : 16
5
For fold : 17
4
For fold : 18
17
For fold : 19
4
For fold : 20
3
For fold : 21
3
For fold : 22
4
For fold : 23
5
For fold : 24
4
For fold : 25
4
For fold : 26
12
For fold : 27
4
For fold : 28
12
For fold : 29
5
For fold : 30
4
The final accuracy is :  0.6777777777777777


In [6]:
#LOW-MEDIUM

CHO_LH_data = CHO_data[CHO_data['Fat'].isin([2.5,18.0])] #considering the two classes for binary classification
CHO_LH_data.loc[CHO_LH_data.Fat == 2.5, 'CHO_bin'] = 0 #giving them binary values, as double variables sometimes create error
CHO_LH_data.loc[CHO_LH_data.Fat== 18.0, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Fat'], axis=1)
#the above 4 lines will be different for different classification, the remaining code will be same
X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009'] #needed for manual loss

final_accuracy =[] #will be used for finding mean across nested cross validation

for i in range(0,30): #as we have 30 IDs
    print("For fold :", i+1) #will print the current fold number, this is for initial check, can delete it
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])] #test data    
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]#train data
    X_train = train.loc[:, train.columns != 'CHO_bin'] #test feature
    Y_train = train.loc[:, train.columns == 'CHO_bin'] #test label
    X_test = test.loc[:, test.columns != 'CHO_bin']#train feature    
    Y_test =test.loc[:, test.columns == 'CHO_bin']#train label
    
    train_ids = X_train['TruncatedID']#needed for inner cross-validation
    train_ids = np.array(train_ids)#needed for inner cross-validation
    train_ids = np.unique(train_ids)#needed for inner cross-validation
    X_train = X_train.drop(['TruncatedID'],axis = 1) #dropping this column as it is not a feature for classification
    X_test = X_test.drop(['TruncatedID'],axis = 1)#dropping this column as it is not a feature for classification
    max_acc = 0 #initializing maximum accuracy which will be needed later    
    ypred_all = []
    for C_val in range(1,20): #ranges of decision tree depth
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification        
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            clf = svm.SVC(C=C_val) #SVM classifier
            #convert dataframe to array
            #xtr_arr = xtr.as_matrix(columns=None)
            #ytr_arr = ytr.as_matrix(columns=None)
            #clf = clf.fit(xtr_arr,ytr_arr)
            clf=clf.fit(xtr,ytr)
            ypred = clf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = C_val
    #best SVM classifier
    print(best_param)
    clf = svm.SVC(C=best_param) 
    clf=clf.fit(X_train,Y_train)
    Y_pred = clf.predict(X_test)
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies in outer CV)

For fold : 1
4
For fold : 2
3
For fold : 3
5
For fold : 4
11
For fold : 5
17
For fold : 6
4
For fold : 7
3
For fold : 8
5
For fold : 9
4
For fold : 10
16
For fold : 11
4
For fold : 12
9
For fold : 13
4
For fold : 14
12
For fold : 15
5
For fold : 16
4
For fold : 17
3
For fold : 18
4
For fold : 19
5
For fold : 20
17
For fold : 21
3
For fold : 22
4
For fold : 23
4
For fold : 24
5
For fold : 25
4
For fold : 26
4
For fold : 27
4
For fold : 28
4
For fold : 29
4
For fold : 30
4
The final accuracy is :  0.65
